In [1]:
import pandas as pd
import paho.mqtt.client as mqtt
import pickle
import json
import string
            
class gSpread(mqtt.Client):
    def __init__(self,broker="op-en.se",port=1883,user="gsheet",passw="celldelning2020",basepath="server/gsheet"):
        super().__init__()
        
        self.dir = {}
        self.topics={}
        self.retained = {}
        self.basepath = basepath
        
        self.debug = True
        
        self.username_pw_set(username=user,password=passw)
        self.connect(broker, port, 60)
        
        self.loop_start()
        self.subscribe("$SYS/broker/log/M/subscribe", 0)
        
        self.lastpayload = ""
        
        self.dataframe = None

    def on_connect(self, mqttc, obj, flags, rc):
        self.subscribe("$SYS/broker/log/M/subscribe", 0)
        print("rc: "+str(rc))

    def on_message(self, mqttc, obj, msg):
        
        try:
        
            if self.debug:
                self.lastpayload = msg.payload

            try:  
                topic = str(msg.payload).split(" ")[-1][:-1]
                client = str(msg.payload).split(" ")[-3]

                if self.debug:
                    print("TOPIC:  " + topic)
                    print("CLIENT:  " + client)

            except:
                pass

            if topic.find(self.basepath)!=0:
                print("Not in basepath")
                return

            #Now the fun begins... 

            parts = topic[len(self.basepath):].split("/")

            if parts[0] =="":
                parts = parts[1:]

            if self.debug:
                print(parts)

            topictype = len(parts)

            dataframe = None

            if parts[0] == "byID":
                if topictype < 3:
                    gid = 0
                else:
                    gid = parts[2]

                dataframe = self.GetDataframe(parts[1],gid)

            print("Dataframed loaded!")

            if self.debug:
                self.dataframe = dataframe

            if topictype < 4:
                self.publish(topic,dataframe.to_json(indent=3))

            if parts[-3] == "cell":
                print("cell")
                
                c = int(parts[-2]) 
                r = int(parts[-1]) 
                        
                data = self.GetCell(dataframe,c,r)
                self.publish(topic,data)
            
            if parts[-2] == "cell":
                
                c = self.column_to_number(parts[-1])
                r = self.get_row(parts[-1])
                
                print(c)
                print(r)
                        
                data = self.GetCell(dataframe,c,r)
                self.publish(topic,data)
                
                
                
        

        except Exception as e: 
            print(e)
            try:
                self.publish(topic,str(e))
            except Exception as e: 
                print(e)
        
    def GetCell(self,dataframe,c,r):
        
            r=r-1
            c=c-1
            
        
            if (c==-1) and (r==-1):
                data = dataframe.index.name
            elif c==-1:
                data = dataframe.index[r]
            elif r==-1:
                data = dataframe.columns[c] 
            else:
                data = dataframe.iloc[r,c]
                
            return data
        
            
    def GetDataframe(self,uid,gid = 0):
        url = 'https://docs.google.com/spreadsheets/d/' + uid + '/export?gid='+ str(gid) +'&format=csv'
    
        if self.debug:
            print(url)
    
        return pd.read_csv(url, index_col=0)
        
    def column_to_number(self,c):
        """Return number corresponding to excel-style column."""
        number=-25
        for l in c:
            if not l in string.ascii_letters:
                continue
            number+=ord(l.upper())-64+25
        return number-1
    
    def get_row(self,r):
        
        row = ""
        
        for l in r:
            if l in string.digits:
                row = row + l
                
        return int(row)-1
        
        
        

In [2]:
sheetserver = gSpread()

rc: 0
TOPIC:  $SYS/broker/log/M/subscribe
CLIENT:  193D2505-F1A9-8610-A172-1A3166B8D343
Not in basepath
TOPIC:  $SYS/broker/log/M/subscribe
CLIENT:  193D2505-F1A9-8610-A172-1A3166B8D343
Not in basepath


In [6]:
sheetserver.dataframe

,uid,population,grundskola,gymnasial,eftergymnasial,uppgiftsaknast,co2_total_2019_ton,co2_total_2019_ton_per_capita,co2_total_2019_ton_per_bil,population.1,...,elbilar_total_2019,elbilar_total_2019_per_capita,elbilar_total_2019_andel_bilar,average driver income_2019,total_driver_education_0,total_driver_education_1,total_driver_education_2,total_driver_education_3,average_driving_distance_2019,average_driving_distance_2019_per_capita
district,,,,,,,,,,,,,,,,,,,,,
Bromma,sthml_dist_0,61000.0,5.2,27.3,65.3,2.2,36466.0200,0.455575,1.429592,80044.0,...,127.0,0.001587,0.004979,593536.30,200,1825,7750,15733,1027.4290,327.4157
Enskede-Årsta-Vantör,sthml_dist_1,96916.0,9.9,34.3,52.7,3.2,39552.4600,0.398521,1.397317,99248.0,...,68.0,0.000685,0.002402,420145.90,368,3676,11574,12688,1009.1590,287.8169
Farsta,sthml_dist_2,45000.0,11.9,37.5,48.3,2.4,24966.9800,0.419726,1.391693,59484.0,...,43.0,0.000723,0.002397,394355.50,132,2654,7395,7759,1004.7390,303.0230
Hägersten-Älvsjö,sthml_dist_3,27500.0,7.7,31.6,58.6,2.2,33619.1500,0.372643,1.429264,90218.0,...,56.0,0.000621,0.002381,459475.60,156,2263,8192,12911,1028.9270,268.2660
Hässelby-Vällingby,sthml_dist_4,67000.0,12.7,38.0,45.5,3.7,37664.5900,0.496214,1.445525,75904.0,...,79.0,0.001041,0.003032,423930.40,383,3308,10961,11404,1056.2270,362.5771
Kungsholmen,sthml_dist_5,70990.0,3.5,22.8,71.0,2.6,21941.8700,0.308211,1.402574,71191.0,...,61.0,0.000857,0.003899,611957.20,106,909,4421,10208,1004.5340,220.7432
Norrmalm,sthml_dist_6,74196.0,3.9,22.0,70.9,3.2,19108.0900,0.277835,1.368480,68775.0,...,54.0,0.000785,0.003867,618318.00,128,800,3878,9157,958.2196,194.5419
Rinkeby-Kista,sthml_dist_7,49273.0,22.0,35.2,35.9,6.9,17015.5700,0.337584,1.468505,50404.0,...,17.0,0.000337,0.001467,314602.20,318,3022,4532,3715,1111.0030,255.4002
Skarpnäcks,sthml_dist_8,46000.0,7.7,28.8,61.3,2.2,17025.5300,0.356638,1.395764,47739.0,...,22.0,0.000461,0.001804,418913.90,106,1295,4074,6723,1010.7030,258.2491


TOPIC:  server/gsheet/byID/1ZGAYgzf_j5ppzmyegupUKVFAGOq_CsyOdav8Ih9gt_8/0/cell/1/1
CLIENT:  mosq-LwVA7IjQHiNPstbT0P
['byID', '1ZGAYgzf_j5ppzmyegupUKVFAGOq_CsyOdav8Ih9gt_8', '0', 'cell', '1', '1']
https://docs.google.com/spreadsheets/d/1ZGAYgzf_j5ppzmyegupUKVFAGOq_CsyOdav8Ih9gt_8/export?gid=0&format=csv
Dataframed loaded!
cell
TOPIC:  test/EMM/5c:cf:7f:2d:26:97/cmd
CLIENT:  5c:cf:7f:2d:26:97
Not in basepath
TOPIC:  test/EMM/5c:cf:7f:2d:26:97/cmd
CLIENT:  5c:cf:7f:2d:26:97
Not in basepath
TOPIC:  test/EMM/5c:cf:7f:2d:26:97/cmd
CLIENT:  5c:cf:7f:2d:26:97
Not in basepath
TOPIC:  test/EMM/5c:cf:7f:2d:26:97/cmd
CLIENT:  5c:cf:7f:2d:26:97
Not in basepath


In [11]:
string.digits

'0123456789'

In [ ]:
myurl

In [ ]:
test = pd.read_csv('https://docs.google.com/spreadsheets/d/' + 
                   '1ZGAYgzf_j5ppzmyegupUKVFAGOq_CsyOdav8Ih9gt_8' +
                   '/export?gid=1905240469&format=csv',
                   # Set first column as rownames in data frame
                   index_col=0,
                   # Parse column values to datetime
                   
                  )
test.head(5)  # Same result as @TomAugspurger

In [ ]:
https://docs.google.com/spreadsheets/d/1ZGAYgzf_j5ppzmyegupUKVFAGOq_CsyOdav8Ih9gt_8/export?format=csv&id=1ZGAYgzf_j5ppzmyegupUKVFAGOq_CsyOdav8Ih9gt_8&gid=1848942430

In [ ]:
def GetDataframe(uid,gid = 0):
    url = 'https://docs.google.com/spreadsheets/d/' + uid + '/export?gid='+ str(gid) +'&format=csv'
    
    return pd.read_csv(url,
                   # Set first column as rownames in data frame
                   index_col=0,
                   # Parse column values to datetime
                   
                  )
    
    

In [ ]:
GetDataframe("1ZGAYgzf_j5ppzmyegupUKVFAGOq_CsyOdav8Ih9gt_8").head()
